In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from ipywidgets import Layout, widgets
from ipywidgets.widgets import Widget
import ipyvuetify as v
from importlib.resources import files
from antakia.data import DimReducMethod
from plotly.graph_objects import FigureWidget, Histogram, Scatter


def widget_at_address(graph : Widget, address : str) :
    """ Returns the widget at the given address in the graph.
        Ex: widget_at_address(graph, "012") returns graph.children[0].children[1].children[2]
        Implemented recursively.
    """
    if len(address) > 1 :
        return widget_at_address(
            graph.children[int(address[0])],
            address[1:]
        )
    else :
        return graph.children[int(address[0])]
        

app_graph = widgets.VBox(
    [
        v.AppBar( # 0
            elevation="4",
            class_="ma-4",
            rounded=True,
            children=[
                v.Layout(
                    children=[
                        widgets.Image(
                            value=open(files("antakia.assets").joinpath("logo_ai-vidence.png"), "rb").read(), 
                            height=str(864 / 20) + "px", 
                            width=str(3839 / 20) + "px"
                        )
                        ],
                    class_="mt-1",
                ),
                v.Html(tag="h2", children=["AntakIA"], class_="ml-3"), # 01
                v.Spacer(),
                v.Btn( # backupBtn # 03 
                    icon=True, children=[v.Icon(children=["mdi-content-save"])], elevation=0
                ),
                v.Btn( # settingsBtn # 04
                    icon=True, children=[v.Icon(children=["mdi-tune"])], elevation=0
                ),
                v.Dialog( # 05
                    children=[
                        v.Card(
                            children=[
                                v.CardTitle(
                                    children=[
                                        v.Icon(class_="mr-5", children=["mdi-cogs"]),
                                        "Settings",
                                    ]
                                ),
                                v.CardText(children=[
                                    v.Row(children=[
                                        v.Slider(
                                            style_="width:20%",
                                            v_model=700,
                                            min=200,
                                            max=1200,
                                            label="With of both scattered plots (in pixels)",
                                        ), 
                                        widgets.IntText(
                                            value="700", disabled=True, layout=Layout(width="40%")
                                        )
                                        ],
                                        ),
                                    ]
                                    ),
                                    ]
                                ),
                            ]
                ),
                v.Btn( # gotoWebBtn # 06
                    icon=True, children=[v.Icon(children=["mdi-web"])], elevation=0
                ),
            ],
        ), 
        widgets.HBox( # 1
            [
                v.Row( # 10
                    class_="ma-3",
                    children=[
                        v.Icon(children=["mdi-numeric-2-box"]),
                        v.Icon(children=["mdi-alpha-d-box"]),
                        v.Switch( # Dim switch # 102
                            class_="ml-3 mr-2",
                            v_model=False,
                            label="",
                        ),
                        v.Icon(children=["mdi-numeric-3-box"]),
                        v.Icon(children=["mdi-alpha-d-box"]),
                    ],
                ),
                v.Layout( # 11
                    class_="pa-2 ma-2",
                    elevation="3",
                        children=[
                                v.Icon( # 110
                                    children=["mdi-format-color-fill"], class_="mt-n5 mr-4"
                                ),
                                v.BtnToggle( # colorChoiceBtnToggle # 111
                                    mandatory=True,
                                    v_model="Y",
                                    children=[
                                        v.Btn( # 1110
                                            icon=True,
                                            children=[v.Icon(children=["mdi-alpha-y-circle-outline"])],
                                            value="y",
                                            v_model=True,
                                        ),
                                        v.Btn( # 1111
                                            icon=True,
                                            children=[v.Icon(children=["mdi-alpha-y-circle"])],
                                            value="y^",
                                            v_model=True,
                                        ),
                                        v.Btn( # app_graph.children[1].children[1].children[1].children[2]
                                            icon=True,
                                            children=[v.Icon(children=["mdi-minus-box-multiple"])],
                                            value="residual",
                                        ),
                                        v.Btn( # app_graph.children[1].children[1].children[1].children[3]
                                            icon=True,
                                            children=[v.Icon(children="mdi-lasso")],
                                            value="current selection",
                                        ),
                                        v.Btn( # app_graph.children[1].children[1].children[1].children[4]
                                            icon=True,
                                            children=[v.Icon(children=["mdi-ungroup"])],
                                            value="regions",
                                        ),
                                        v.Btn( # app_graph.children[1].children[1].children[1].children[5]
                                            icon=True,
                                            children=[v.Icon(children=["mdi-select-off"])],
                                            value="not selected",
                                        ),
                                        v.Btn( # app_graph.children[1].children[1].children[1].children[6]
                                            icon=True,
                                            children=[v.Icon(children=["mdi-star"])],
                                            value="auto",
                                        ),
                                    ],
                                ),
                                v.Btn( # opacityBtn # 112
                                    icon=True,
                                    children=[v.Icon(children=["mdi-opacity"])],
                                    class_="ma-2 ml-6 pa-3",
                                    elevation="3",
                                ),
                                v.Select( # explanationSelect # 113
                                    label="Explanation method",
                                    items=[
                                        {'text': "SHAP (imported)", 'disabled': True },
                                        {'text': "SHAP (computed)", 'disabled': True },
                                        {'text': "LIME (imported)", 'disabled': True },
                                        {'text': "LIME (computed)", 'disabled': True }
                                        ],
                                    class_="ma-2 mt-1 ml-6",
                                    style_="width: 150px",
                                    disabled = False,
                                )
                            ],
                ),
                v.Layout( # 12
                    class_="mt-3",
                    children=[
                        widgets.HBox( # 120
                            [
                                v.Select( # projSelectVS # 1200
                                    label="Projection in the VS :",
                                    items=DimReducMethod.getDimReducMethodsAsStrList(),
                                    style_="width: 150px",
                                ),
                                v.Layout( # 1201
                                    children=[
                                        v.Menu( # projSettingsMenuVS # 12010
                                            v_slots=[
                                            {
                                                "name": "activator",
                                                "variable": "props",
                                                "children": v.Btn(
                                                    v_on="props.on",
                                                    icon=True,
                                                    size="x-large",
                                                    children=[v.Icon(children=["mdi-cogs"], size="large")],
                                                    class_="ma-2 pa-3",
                                                    elevation="3",
                                                ),
                                            }
                                            ],
                                            children=[
                                                v.Card( # app_graph.children[1].children[2].children[0].children[1].children[0].childen[0] 
                                                    class_="pa-4",
                                                    rounded=True,
                                                    children=[
                                                        widgets.VBox([ # ProjVS sliders
                                                            v.Slider(
                                                                v_model=10, min=5, max=30, step=1, label="Number of neighbours"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            v.Slider(
                                                                v_model=0.5, min=0.1, max=0.9, step=0.1, label="MN ratio"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            v.Slider(
                                                                v_model=2, min=0.1, max=5, step=0.1, label="FP ratio"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            ],
                                                        )
                                                        ],
                                                    min_width="500",
                                                )
                                            ],
                                        v_model=False,
                                        close_on_content_click=False,
                                        offset_y=True,
                                        )
                                    ]
                                ),
                                widgets.HBox( # app_graph.children[1].children[2].children[0].children[2]
                                    [ 
                                    v.ProgressCircular( # VSBusyBox # app_graph.children[1].children[2].children[0].children[2].children[0]
                                        indeterminate=True, color="blue", width="6", size="35", class_="mx-4 my-3"
                                    )
                                    ]),
                            ]
                        ),
                        widgets.HBox(  # 121
                            [
                                v.Select( # projSelectES # 121 0
                                    label="Projection in the ES :",
                                    items=DimReducMethod.getDimReducMethodsAsStrList(),
                                    style_="width: 150px",
                                ),
                                v.Layout( # 1211
                                    children=[
                                        v.Menu( # projSettingsMenuES # 121 10
                                            v_slots=[
                                                {
                                                    "name": "activator",
                                                    "variable": "props",
                                                    "children": v.Btn(
                                                        v_on="props.on",
                                                        icon=True,
                                                        size="x-large",
                                                        children=[v.Icon(children=["mdi-cogs"], size="large")],
                                                        class_="ma-2 pa-3",
                                                        elevation="3",
                                                    ),
                                                }
                                            ],
                                            children=[
                                                v.Card(
                                                    class_="pa-4",
                                                    rounded=True,
                                                    children=[
                                                        widgets.VBox([ # ProjES sliders
                                                            v.Slider(
                                                                v_model=10, min=5, max=30, step=1, label="Number of neighbours"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            v.Slider(
                                                                v_model=0.5, min=0.1, max=0.9, step=0.1, label="MN ratio"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            v.Slider(
                                                                v_model=2, min=0.1, max=5, step=0.1, label="FP ratio"
                                                            ),
                                                            v.Html(class_="ml-3", tag="h3", children=["#"]),
                                                            ],
                                                        )
                                                        ],
                                                    min_width="500",
                                                )
                                            ],
                                            v_model=False,
                                            close_on_content_click=False,
                                            offset_y=True,
                                        )
                                    ]),
                                widgets.HBox( # 121 2
                                    [
                                    v.ProgressCircular( # ESBusyBox # 121 20
                                        indeterminate=True, color="blue", width="6", size="35", class_="mx-4 my-3"
                                    )
                                    ]),
                            ]
                        ),
                    ],
                )
            ],
            layout=Layout(
                width="100%",
                display="flex",
                flex_flow="row",
                justify_content="space-around",
            ),
        ),
        widgets.VBox( # 2
            [
            widgets.HBox( # 20
                [
                widgets.VBox( # 200
                        [
                        widgets.HTML("<h3>Values Space<h3>"), # 2000
                        FigureWidget( # 2001
                            data=Scatter(
                                x=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD')),
                                y=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                                mode="markers", 
                                marker= dict( 
                                    color=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                                    colorscale="Viridis",
                                    colorbar=dict(
                                        title="y",
                                        thickness=20,
                                    ),
                                ), 
                                customdata=dict( 
                                    color=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                                    colorscale="Viridis",
                                    colorbar=dict(
                                        title="y",
                                        thickness=20,
                                    ),
                                )["color"],
                                hovertemplate = '%{customdata:.3f}')
                        )
                        ],
                        layout=Layout(
                            display="flex", align_items="center", margin="0px 0px 0px 0px"
                        ),
                )
                ,
                widgets.VBox( #  #201
                        [
                        widgets.HTML("<h3>Explanations Space<h3>"), # 201 0
                        FigureWidget( # 201 1
                            data=Scatter(
                                x=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD')),
                                y=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                            mode="markers", 
                            marker=
                                dict( 
                                    color=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                                    colorscale="Viridis"
                                ), 
                            customdata=dict( 
                                color=pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))['A'],
                                colorscale="Viridis",
                                colorbar=dict(
                                    title="y",
                                    thickness=20,
                                ),
                                )["color"],
                            hovertemplate = '%{customdata:.3f}')
                        )
                        ],
                        layout=Layout(
                            display="flex", align_items="center", margin="0px 0px 0px 0px"
                        )
                )
                ],
                layout=Layout(width="100%")
                )
        ]    
        ),
        v.Container( # app_graph.children[3]
            fluid = True,
            children=[
                v.Tabs( # app_graph.children[3].children[0]
                    v_model=0, # default active tab
                    children=
                    [
                        v.Tab(children=["1. Selection"]),  # 300
                        v.Tab(children=["2. Refinement"]), # 301
                        v.Tab(children=["3. Sub-model"]), # 302
                        v.Tab(children=["4. Regions"]) # 303
                    ] 
                    + 
                    [
                        v.TabItem(  # Tab 1) Selection # 304
                            children=[
                                v.Card( # selectionCard # 304 0
                                    class_="ma-2",
                                    elevation=0,
                                    children=[
                                        v.Layout(
                                            children=[
                                                v.Icon(children=["mdi-lasso"]),
                                                v.Html(
                                                    class_="mt-2 ml-4",
                                                    tag="h4",
                                                    children=[
                                                        "0 point selected : use the lasso tool on the figures above or use the auto-selection tool below"
                                                    ],
                                                ),
                                            ]
                                        ),
                                    ],
                                ),
                                v.ExpansionPanels( # out_accordion # 304 1
                                    class_="ma-2",
                                    children=[
                                        v.ExpansionPanel(
                                            children=[
                                                v.ExpansionPanelHeader(children=["Data selected"]),
                                                v.ExpansionPanelContent(
                                                    children=[
                                                    v.Alert(
                                                        max_height="400px",
                                                        style_="overflow: auto",
                                                        elevation="0",
                                                        children=[
                                                            v.Row(
                                                                class_="d-flex flex-row justify-space-between",
                                                                children=[
                                                                    v.Layout(
                                                                        style_="min-width: 47%; max-width: 47%",
                                                                        children=[
                                                                            v.Html(
                                                                                tag="h4",
                                                                                children=["Select points on the figure to see their values ​​here"],
                                                                            )
                                                                        ],
                                                                    ),
                                                                    v.Divider(class_="ma-2", vertical=True),
                                                                    v.Layout(
                                                                        style_="min-width: 47%; max-width: 47%",
                                                                        children=[
                                                                            v.Html(
                                                                                tag="h4",
                                                                                children=[
                                                                                    "Select points on the figure to see their SHAP values ​​here"
                                                                                ],
                                                                            )
                                                                        ],
                                                                    ),
                                                                ],
                                                            ),
                                                        ],
                                                    ),
                                                    ]),
                                            ]
                                        )
                                    ],
                                ),
                                v.Layout( # clusterGrp # 304 2
                                    class_="d-flex flex-row",
                                    children=[
                                        v.Btn( # 304 20
                                            class_="ma-1 mt-2 mb-0",
                                            elevation="2",
                                            children=[v.Icon(children=["mdi-magnify"]), "Find clusters"],
                                        ),
                                        v.Checkbox( # 304 21
                                            v_model=True, label="Optimal number of clusters :", class_="ma-3"
                                        ),
                                        v.Slider( # 304 22
                                            style_="width : 30%",
                                            class_="ma-3 mb-0",
                                            min=2,
                                            max=20,
                                            step=1,
                                            v_model=3,
                                            disabled=True,
                                        ),
                                        v.Html( # 304 23
                                            tag="h3",
                                            class_="ma-3 mb-0",
                                            children=["Number of clusters #"],
                                        ),
                                    ],
                                ),
                                v.ProgressLinear( # loadingClustersProgLinear # 304 3
                                    indeterminate=True, class_="ma-3", style_="width : 100%"
                                ),
                                v.Row( # clusterResults # 304 4
                                    children=[
                                        v.Layout( #  304 40
                                            class_="flex-grow-0 flex-shrink-0", children=[
                                                v.RadioGroup(
                                                    v_model=None,
                                                    class_="mt-10 ml-7",
                                                    style_="width : 10%",
                                                    children=[
                                                        "1", "2", "3"
                                                    ],
                                                ),
                                                ]
                                        ),
                                        v.Layout( #  304 41
                                            class_="flex-grow-0 flex-shrink-0", children=[
                                                v.Col(
                                                    class_="mt-10 mb-2 ml-0 d-flex flex-column justify-space-between",
                                                    style_="width : 10%",
                                                    children=[
                                                        "4", "5", "6"
                                                    ],
                                                )
                                                ]
                                        ),
                                        v.Layout( #  304 42
                                            class_="flex-grow-1 flex-shrink-0",
                                            children=[
                                                v.DataTable(
                                                    class_="w-100",
                                                    style_="width : 100%",
                                                    v_model=[],
                                                    show_select=False,
                                                    # headers=columns,
                                                    # explanationsMenuDict=new_df.to_dict("records"),
                                                    item_value="Region #",
                                                    item_key="Region #",
                                                    hide_default_footer=True,
                                                ),
                                                ],
                                        ),
                                    ],
                                ),
                                v.Layout( # clusterResults #  304 5
                                    class_="d-flex flex-row justify-center align-center",
                                    children=[
                                        v.Spacer(),
                                            v.Btn(
                                                class_="ma-3",
                                                children=[
                                                    v.Icon(children=["mdi-creation"], class_="mr-3"),
                                                    "Magic button",
                                                ],
                                        ),
                                        v.Checkbox(v_model=True, label="Demonstration mode", class_="ma-4"),
                                        v.TextField(
                                            class_="shrink",
                                            type="number",
                                            label="Time between the steps (ds)",
                                            v_model=10,
                                        ),
                                        v.Spacer(),
                                    ],
                                )
                            ]
                        ), 
                        v.TabItem( # Tab 2) Refinement # 305
                            children=[
                                v.Col( # 305 0
                                    children=[
                                        widgets.VBox( # skopeBtnsGrp # 305 00
                                            [
                                            v.Layout( # skopeBtns # 305 000
                                                class_="d-flex flex-row",
                                                children=[
                                                    v.Btn( # validateSkopeBtn # 305 000 0
                                                        class_="ma-1",
                                                        children=[
                                                            v.Icon(class_="mr-2", children=["mdi-auto-fix"]),
                                                            "Skope-Rules",
                                                        ],
                                                    ),
                                                    v.Btn( # reinitSkopeBtn # 305 000 1
                                                        class_="ma-1",
                                                        children=[
                                                            v.Icon(class_="mr-2", children=["mdi-skip-backward"]),
                                                            "Come back to the initial rules",
                                                        ],
                                                    ),
                                                    v.Spacer(), # 305 000 2
                                                    v.Checkbox( # beeSwarmCheck # 305 000 3
                                                        v_model=True,
                                                        label="Show Shapley's beeswarm plots",
                                                        class_="ma-1 mr-3",
                                                    )
                                                    ,
                                                ],
                                            ),
                                            v.Layout( # skopeText # skopeBtns # 305 001
                                                class_="d-flex flex-row",
                                                children=[
                                                    v.Card( # ourVSSkopeText # skopeBtns # 305 001 0
                                                        style_="width: 50%;",
                                                        class_="ma-3",
                                                        children=[
                                                            v.Row(  # 30500100
                                                                class_="ml-4",
                                                                children=[
                                                                    v.Icon(children=["mdi-target"]), # 305 010 00
                                                                    v.CardTitle(children=["Rules applied to the Values Space"]), # 305 010 01
                                                                    v.Spacer(), # 305 010 02
                                                                    v.Html( # 305 010 03
                                                                        class_="mr-5 mt-5 font-italic",
                                                                        tag="p",
                                                                        children=["precision = /"],
                                                                    ),
                                                                ],
                                                            ),
                                                            v.Card( # ourVSSkopeCard # 305 001 01
                                                                class_="mx-4 mt-0",
                                                                elevation=0,
                                                                children=[
                                                                    v.CardText(
                                                                        children=[
                                                                            v.Row(
                                                                                class_="font-weight-black text-h5 mx-10 px-10 d-flex flex-row justify-space-around",
                                                                                children=[
                                                                                    "Waiting for the skope-rules to be applied...",
                                                                                ],
                                                                            )
                                                                        ]
                                                                    )
                                                                ],
                                                            )
                                                        ],
                                                    ),
                                                    v.Card( # ourESSkopeText # 305 001 1
                                                        style_="width: 50%;",
                                                        class_="ma-3",
                                                        children=[
                                                            v.Row( # 305 001 10
                                                                class_="ml-4",
                                                                children=[
                                                                    v.Icon(children=["mdi-target"]),
                                                                    v.CardTitle(children=["Rules applied on the Explanatory Space"]), 
                                                                    v.Spacer(),
                                                                    v.Html(
                                                                        class_="mr-5 mt-5 font-italic",
                                                                        tag="p",
                                                                        children=["precision = /"],
                                                                    ),
                                                                ],
                                                            ),
                                                            v.Card( # ourESSkopeCard # 305 001 11
                                                                class_="mx-4 mt-0",
                                                                elevation=0,
                                                                # style_="width: 100%;",
                                                                children=[
                                                                    v.CardText(
                                                                        children=[
                                                                            v.Row(
                                                                                class_="font-weight-black text-h5 mx-10 px-10 d-flex flex-row justify-space-around",
                                                                                children=[
                                                                                    "Waiting for the Skope-rules to be applied...",
                                                                                ],
                                                                            )
                                                                        ]
                                                                    ),
                                                                ],
                                                            ),
                                                        ],
                                                    )
                                                ]
                                            ) # End v.Layout / skopeText
                                        ]
                                        ), # End VBox / skopeBtnsGrp
                                        widgets.VBox( # skopeAccordion # 305 01
                                            children=[ # RuleVariableRefiner objects are inserted here by the app
                                            ],
                                            layout=Layout(width="100%", height="auto"),
                                        ), # End of VBox 30501
                                        v.Row( #addButtonsGrp # 305 02
                                            children=[
                                                v.Btn( # 305 020
                                                    class_="ma-4 pa-2 mb-1",
                                                    children=[v.Icon(children=["mdi-plus"]), "Add a rule"],
                                                ), 
                                                v.Select( # addAnotherFeatureWgt # 305 021
                                                    class_="mr-3 mb-0",
                                                    explanationsMenuDict=["/"],
                                                    v_model="/",
                                                    style_="max-width : 15%",
                                                ), 
                                                v.Spacer(), # 305 022
                                                v.Btn( # 305 023
                                                    class_="ma-4 pa-2 mb-1",
                                                    children=[v.Icon(class_="mr-4", children=["mdi-map"]), "Display the map"],
                                                    color="white",
                                                    disabled=True,
                                                ),
                                                ]
                                        ),
                                        ]
                                )
                            ]
                        ), 
                        v.TabItem( # Tab 3 # 306
                            children=[
                                    widgets.VBox( # 306 0
                                        [
                                            v.ProgressLinear( # loadingModelsProgLinear # 306 00
                                                indeterminate=True,
                                                class_="my-0 mx-15",
                                                style_="width: 100%;",
                                                color="primary",
                                                height="5",
                                            ), 
                                            v.SlideGroup( # subModelslides # 306 01
                                                v_model=None,
                                                class_="ma-3 pa-3",
                                                elevation=4,
                                                center_active=True,
                                                show_arrows=True,
                                                children=
                                                [
                                                    v.SlideItem( # 306 010 # dummy SlideItem. Will be replaced by the app
                                                        # style_="width: 30%",
                                                        children=[
                                                            v.Card(
                                                                class_="grow ma-2",
                                                                children=[
                                                                    v.Row(
                                                                        class_="ml-5 mr-4",
                                                                        children=[
                                                                            v.Icon(children=["a name"]),
                                                                            v.CardTitle(
                                                                                children=["model foo"]
                                                                            ),
                                                                        ],
                                                                    ),
                                                                    v.CardText(
                                                                        class_="mt-0 pt-0",
                                                                        children=["Model's score"],
                                                                    ),
                                                                ],
                                                            )
                                                        ],
                                                    )
                                                ],
                                            ),
                                            ]
                                    )
                                ]
                        ),
                        v.TabItem( # Tab 4 # 307
                            children=[
                                v.Col(
                                children=[
                                    widgets.VBox(
                                        [
                                            ]
                                    ),         
                                ]
                                )
                            ]
                        )
                    ]
                )
            ],
            class_="mt-0",
            outlined=True
        )
    ]
)

display(app_graph)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
type(widget_at_address(app_graph, "30502"))

ipyvuetify.generated.Row.Row